# USER O 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Carregar o dataframe inicial
User = pd.read_excel('EVIO_history_01-02-2023_29-02-2024.xlsx')
User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)', 'Nº cartão EVIO']]

# Remover valores de carregamento de energia inferiores a 1 kWh
User = User[User['Total Energy (kWh)'] >= 1]

# Remover valores de duração de carregamento inferiores a 5 minutos
User = User[User['Duration (min)'] >= 5]

User = User[User['Nº cartão EVIO'] == 0]

# Convertendo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'], format='%m/%d/%Y | %H:%M')
User['Stop date'] = pd.to_datetime(User['Stop date'], format='%m/%d/%Y | %H:%M')

# Criando uma nova coluna 'Weekday' que contém o dia da semana
User['Weekday'] = User['Start date'].dt.day_name()

print(User)

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

User['Weekday'] = User['Weekday'].map(weekday_mapping)

print(User)

              Start date           Stop date  Duration (min)  \
31   2024-02-25 10:20:00 2024-02-25 10:37:00          16.500   
35   2024-02-23 08:37:00 2024-02-23 11:31:00         174.000   
52   2024-02-20 14:12:00 2024-02-20 20:23:00         370.850   
61   2024-02-19 08:31:00 2024-02-19 11:39:00         188.083   
91   2024-02-11 10:48:00 2024-02-11 11:14:00          25.983   
...                  ...                 ...             ...   
1536 2023-04-03 08:39:00 2023-04-03 13:38:00         298.967   
1542 2023-03-31 08:48:00 2023-03-31 13:36:00         288.500   
1545 2023-03-31 08:41:00 2023-03-31 10:26:00         105.350   
1548 2023-03-31 08:37:00 2023-03-31 08:45:00           8.283   
1559 2023-03-29 08:35:00 2023-03-29 10:26:00         111.117   

      Total Energy (kWh) Nº cartão EVIO    Weekday  
31                 28.52              0     Sunday  
35                 31.10              0     Friday  
52                 27.23              0    Tuesday  
61                 

In [2]:
# Convertendo as colunas de data para o tipo datetime
#User['Start date'] = pd.to_datetime(User['Start date'])

# Criar uma nova coluna contendo apenas a data (sem a hora)
User['Date'] = User['Start date'].dt.date

# Obter o intervalo de data
start_date = User['Date'].min()
end_date = User['Date'].max()

# Criar um novo DataFrame com todas as datas no intervalo
date_range = pd.date_range(start=start_date, end=end_date)

# Criar um DataFrame vazio para armazenar as datas sem carregamento
empty_dates = pd.DataFrame({'Date': date_range})

#print(User['Date'])
#print(empty_dates, User)

User['Date'] = pd.to_datetime(User['Date'], format='%Y/%m/%d')

#print(empty_dates, User)

"""print(User['Date'].dtype)
print(User.dtype)
print(empty_dates['Date'].dtype)
print(empty_dates.dtype)
"""
# Mesclar os dois DataFrames usando apenas a coluna 'Date'
merged_df = pd.merge(empty_dates, User, on='Date', how='left')

# Preencher NaN e NaT por 0 em todo o DataFrame
merged_df = merged_df.fillna(0)


merged_df['Weekday'] = merged_df['Date'].dt.day_name()

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

merged_df['Weekday'] = merged_df['Weekday'].map(weekday_mapping)

print(merged_df)

          Date           Start date            Stop date  Duration (min)  \
0   2023-03-29  2023-03-29 08:35:00  2023-03-29 10:26:00         111.117   
1   2023-03-30                    0                    0           0.000   
2   2023-03-31  2023-03-31 08:48:00  2023-03-31 13:36:00         288.500   
3   2023-03-31  2023-03-31 08:41:00  2023-03-31 10:26:00         105.350   
4   2023-03-31  2023-03-31 08:37:00  2023-03-31 08:45:00           8.283   
..         ...                  ...                  ...             ...   
339 2024-02-21                    0                    0           0.000   
340 2024-02-22                    0                    0           0.000   
341 2024-02-23  2024-02-23 08:37:00  2024-02-23 11:31:00         174.000   
342 2024-02-24                    0                    0           0.000   
343 2024-02-25  2024-02-25 10:20:00  2024-02-25 10:37:00          16.500   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0                 12.60             

In [3]:
print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)


# Convertendo as colunas Start date e Stop date para datetime, se necessário
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')


print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)

print(merged_df)


# Criar colunas para cada hora do dia (8h às 20h)
hours = range(8, 21)
for hour in hours:
    merged_df[f'Charging_{hour}h'] = 0

# Preencher as colunas com valores binários (1 ou 0) apenas para as datas presentes no DataFrame original
for index, row in merged_df.iterrows():
    if not pd.isnull(row['Start date']) and not pd.isnull(row['Stop date']):
        start_hour = row['Start date'].hour
        stop_hour = row['Stop date'].hour
        for hour in range(8, 21):
            if hour >= start_hour and hour <= stop_hour:
                merged_df.at[index, f'Charging_{hour}h'] = 1

print(merged_df)

object
object
datetime64[ns]
datetime64[ns]
          Date          Start date           Stop date  Duration (min)  \
0   2023-03-29 2023-03-29 08:35:00 2023-03-29 10:26:00         111.117   
1   2023-03-30                 NaT                 NaT           0.000   
2   2023-03-31 2023-03-31 08:48:00 2023-03-31 13:36:00         288.500   
3   2023-03-31 2023-03-31 08:41:00 2023-03-31 10:26:00         105.350   
4   2023-03-31 2023-03-31 08:37:00 2023-03-31 08:45:00           8.283   
..         ...                 ...                 ...             ...   
339 2024-02-21                 NaT                 NaT           0.000   
340 2024-02-22                 NaT                 NaT           0.000   
341 2024-02-23 2024-02-23 08:37:00 2024-02-23 11:31:00         174.000   
342 2024-02-24                 NaT                 NaT           0.000   
343 2024-02-25 2024-02-25 10:20:00 2024-02-25 10:37:00          16.500   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0               

In [6]:
# Ordenar o dataframe pela coluna 'Start date' para garantir que os dados estejam em ordem temporal
merged_df.sort_values(by='Date', inplace=True)


# Converter colunas de data para datetime, tratando '0' como NaT
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

# Atualizar Start date e Stop date onde Total Energy é igual a zero
mask = merged_df['Total Energy (kWh)'] == 0
merged_df.loc[mask, 'Start date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)
merged_df.loc[mask, 'Stop date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)

# Calcular a diferença em horas desde o último carregamento, ignorando os valores NaT
merged_df['Hours_since_last_charge'] = merged_df['Start date'].diff().dt.total_seconds() / 3600

# Substituir NaN com 0 (ou outro valor, se necessário)
merged_df['Hours_since_last_charge'].fillna(0, inplace=True)

# Mostrar o DataFrame atualizado
print(merged_df)

          Date          Start date           Stop date  Duration (min)  \
0   2023-03-29 2023-03-29 08:35:00 2023-03-29 10:26:00         111.117   
1   2023-03-30 2023-03-30 00:00:00 2023-03-30 00:00:00           0.000   
2   2023-03-31 2023-03-31 08:48:00 2023-03-31 13:36:00         288.500   
3   2023-03-31 2023-03-31 08:41:00 2023-03-31 10:26:00         105.350   
4   2023-03-31 2023-03-31 08:37:00 2023-03-31 08:45:00           8.283   
..         ...                 ...                 ...             ...   
339 2024-02-21 2024-02-21 00:00:00 2024-02-21 00:00:00           0.000   
340 2024-02-22 2024-02-22 00:00:00 2024-02-22 00:00:00           0.000   
341 2024-02-23 2024-02-23 08:37:00 2024-02-23 11:31:00         174.000   
342 2024-02-24 2024-02-24 00:00:00 2024-02-24 00:00:00           0.000   
343 2024-02-25 2024-02-25 10:20:00 2024-02-25 10:37:00          16.500   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
0                 12.60          

In [7]:
# Converter colunas de data para datetime, tratando '0' como NaT
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

# Atualizar Start date e Stop date onde Total Energy é igual a zero
mask = merged_df['Total Energy (kWh)'] == 0
merged_df.loc[mask, 'Start date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)
merged_df.loc[mask, 'Stop date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)

# Calcular a diferença em horas desde o último carregamento
hours_since_last_charge = []
previous_stop_date = None

for index, row in merged_df.iterrows():
    if pd.isnull(row['Start date']) or pd.isnull(row['Stop date']):
        hours_since_last_charge.append(0)
    else:
        if previous_stop_date is None:
            hours_since_last_charge.append(0)
        else:
            diff = (row['Start date'] - previous_stop_date).total_seconds() / 3600
            hours_since_last_charge.append(max(diff, 0))
    previous_stop_date = row['Stop date']

merged_df['Hours_since_last_charge'] = hours_since_last_charge

# Mostrar o DataFrame atualizado
print(merged_df)

          Date          Start date           Stop date  Duration (min)  \
0   2023-03-29 2023-03-29 08:35:00 2023-03-29 10:26:00         111.117   
1   2023-03-30 2023-03-30 00:00:00 2023-03-30 00:00:00           0.000   
2   2023-03-31 2023-03-31 08:48:00 2023-03-31 13:36:00         288.500   
3   2023-03-31 2023-03-31 08:41:00 2023-03-31 10:26:00         105.350   
4   2023-03-31 2023-03-31 08:37:00 2023-03-31 08:45:00           8.283   
..         ...                 ...                 ...             ...   
339 2024-02-21 2024-02-21 00:00:00 2024-02-21 00:00:00           0.000   
340 2024-02-22 2024-02-22 00:00:00 2024-02-22 00:00:00           0.000   
341 2024-02-23 2024-02-23 08:37:00 2024-02-23 11:31:00         174.000   
342 2024-02-24 2024-02-24 00:00:00 2024-02-24 00:00:00           0.000   
343 2024-02-25 2024-02-25 10:20:00 2024-02-25 10:37:00          16.500   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
0                 12.60          

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Definir o range de horas de interesse
#hours = range(8, 21)

# Preparar a matriz de entrada (X) e saída (y)
X = merged_df[['Weekday', 'Hours_since_last_charge']].values
y = merged_df[[f'Charging_{hour}h' for hour in hours]].values

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in hours])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões
print("Previsões:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.ravel(), predictions.ravel())

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')

Acurácia do modelo: 0.919732441471572
Previsões:
     Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
1   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
2   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
3   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
4   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
..           ...           ...           ...           ...           ...   
64  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
65  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
66  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
67  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
68  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   

    Charging_13h  Charging_14h  Chargi

# User 29

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Carregar o dataframe inicial
User = pd.read_excel('EVIO_history_01-02-2023_29-02-2024.xlsx')
User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)', 'Nº cartão EVIO']]

# Remover valores de carregamento de energia inferiores a 1 kWh
User = User[User['Total Energy (kWh)'] >= 1]

# Remover valores de duração de carregamento inferiores a 5 minutos
User = User[User['Duration (min)'] >= 5]

User = User[User['Nº cartão EVIO'] == 29]

# Convertendo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'], format='%m/%d/%Y | %H:%M')
User['Stop date'] = pd.to_datetime(User['Stop date'], format='%m/%d/%Y | %H:%M')

# Criando uma nova coluna 'Weekday' que contém o dia da semana
User['Weekday'] = User['Start date'].dt.day_name()

print(User)

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

User['Weekday'] = User['Weekday'].map(weekday_mapping)

print(User)

              Start date           Stop date  Duration (min)  \
13   2024-02-28 08:36:00 2024-02-28 13:27:00         290.983   
66   2024-02-16 14:15:00 2024-02-16 15:28:00          72.617   
67   2024-02-16 13:48:00 2024-02-16 14:14:00          25.583   
111  2024-02-06 08:34:00 2024-02-06 13:06:00         272.633   
127  2024-02-01 12:24:00 2024-02-01 12:44:00          20.433   
...                  ...                 ...             ...   
1708 2023-02-15 13:53:00 2023-02-15 19:29:00         336.067   
1724 2023-02-14 08:38:00 2023-02-14 12:48:00         250.700   
1730 2023-02-13 08:31:00 2023-02-13 12:59:00         268.833   
1762 2023-02-03 08:08:00 2023-02-03 10:31:00         143.100   
1771 2023-02-01 08:50:00 2023-02-01 12:42:00         232.767   

      Total Energy (kWh) Nº cartão EVIO    Weekday  
13                52.820             29  Wednesday  
66                12.860             29     Friday  
67                33.580             29     Friday  
111               4

In [3]:
# Convertendo as colunas de data para o tipo datetime
#User['Start date'] = pd.to_datetime(User['Start date'])

# Criar uma nova coluna contendo apenas a data (sem a hora)
User['Date'] = User['Start date'].dt.date

# Obter o intervalo de data
start_date = User['Date'].min()
end_date = User['Date'].max()

# Criar um novo DataFrame com todas as datas no intervalo
date_range = pd.date_range(start=start_date, end=end_date)

# Criar um DataFrame vazio para armazenar as datas sem carregamento
empty_dates = pd.DataFrame({'Date': date_range})

#print(User['Date'])
#print(empty_dates, User)

User['Date'] = pd.to_datetime(User['Date'], format='%Y/%m/%d')

#print(empty_dates, User)

"""print(User['Date'].dtype)
print(User.dtype)
print(empty_dates['Date'].dtype)
print(empty_dates.dtype)
"""
# Mesclar os dois DataFrames usando apenas a coluna 'Date'
merged_df = pd.merge(empty_dates, User, on='Date', how='left')

# Preencher NaN e NaT por 0 em todo o DataFrame
merged_df = merged_df.fillna(0)


merged_df['Weekday'] = merged_df['Date'].dt.day_name()

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

merged_df['Weekday'] = merged_df['Weekday'].map(weekday_mapping)

print(merged_df)

          Date           Start date            Stop date  Duration (min)  \
0   2023-02-01  2023-02-01 08:50:00  2023-02-01 12:42:00         232.767   
1   2023-02-02                    0                    0           0.000   
2   2023-02-03  2023-02-03 08:08:00  2023-02-03 10:31:00         143.100   
3   2023-02-04                    0                    0           0.000   
4   2023-02-05                    0                    0           0.000   
..         ...                  ...                  ...             ...   
405 2024-02-24                    0                    0           0.000   
406 2024-02-25                    0                    0           0.000   
407 2024-02-26                    0                    0           0.000   
408 2024-02-27                    0                    0           0.000   
409 2024-02-28  2024-02-28 08:36:00  2024-02-28 13:27:00         290.983   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0                 41.44             

In [4]:
print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)


# Convertendo as colunas Start date e Stop date para datetime, se necessário
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')


print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)

print(merged_df)


# Criar colunas para cada hora do dia (8h às 20h)
hours = range(8, 21)
for hour in hours:
    merged_df[f'Charging_{hour}h'] = 0

# Preencher as colunas com valores binários (1 ou 0) apenas para as datas presentes no DataFrame original
for index, row in merged_df.iterrows():
    if not pd.isnull(row['Start date']) and not pd.isnull(row['Stop date']):
        start_hour = row['Start date'].hour
        stop_hour = row['Stop date'].hour
        for hour in range(8, 21):
            if hour >= start_hour and hour <= stop_hour:
                merged_df.at[index, f'Charging_{hour}h'] = 1

print(merged_df)

object
object
datetime64[ns]
datetime64[ns]
          Date          Start date           Stop date  Duration (min)  \
0   2023-02-01 2023-02-01 08:50:00 2023-02-01 12:42:00         232.767   
1   2023-02-02                 NaT                 NaT           0.000   
2   2023-02-03 2023-02-03 08:08:00 2023-02-03 10:31:00         143.100   
3   2023-02-04                 NaT                 NaT           0.000   
4   2023-02-05                 NaT                 NaT           0.000   
..         ...                 ...                 ...             ...   
405 2024-02-24                 NaT                 NaT           0.000   
406 2024-02-25                 NaT                 NaT           0.000   
407 2024-02-26                 NaT                 NaT           0.000   
408 2024-02-27                 NaT                 NaT           0.000   
409 2024-02-28 2024-02-28 08:36:00 2024-02-28 13:27:00         290.983   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0               

In [5]:
# Ordenar o dataframe pela coluna 'Start date' para garantir que os dados estejam em ordem temporal
merged_df.sort_values(by='Date', inplace=True)


# Converter colunas de data para datetime, tratando '0' como NaT
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

# Atualizar Start date e Stop date onde Total Energy é igual a zero
mask = merged_df['Total Energy (kWh)'] == 0
merged_df.loc[mask, 'Start date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)
merged_df.loc[mask, 'Stop date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)

# Calcular a diferença em horas desde o último carregamento, ignorando os valores NaT
merged_df['Hours_since_last_charge'] = merged_df['Start date'].diff().dt.total_seconds() / 3600

# Substituir NaN com 0 (ou outro valor, se necessário)
merged_df['Hours_since_last_charge'].fillna(0, inplace=True)

# Mostrar o DataFrame atualizado
print(merged_df)

          Date          Start date           Stop date  Duration (min)  \
0   2023-02-01 2023-02-01 08:50:00 2023-02-01 12:42:00         232.767   
1   2023-02-02 2023-02-02 00:00:00 2023-02-02 00:00:00           0.000   
2   2023-02-03 2023-02-03 08:08:00 2023-02-03 10:31:00         143.100   
3   2023-02-04 2023-02-04 00:00:00 2023-02-04 00:00:00           0.000   
4   2023-02-05 2023-02-05 00:00:00 2023-02-05 00:00:00           0.000   
..         ...                 ...                 ...             ...   
405 2024-02-24 2024-02-24 00:00:00 2024-02-24 00:00:00           0.000   
406 2024-02-25 2024-02-25 00:00:00 2024-02-25 00:00:00           0.000   
407 2024-02-26 2024-02-26 00:00:00 2024-02-26 00:00:00           0.000   
408 2024-02-27 2024-02-27 00:00:00 2024-02-27 00:00:00           0.000   
409 2024-02-28 2024-02-28 08:36:00 2024-02-28 13:27:00         290.983   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
0                 41.44          

In [6]:
# Converter colunas de data para datetime, tratando '0' como NaT
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

# Atualizar Start date e Stop date onde Total Energy é igual a zero
mask = merged_df['Total Energy (kWh)'] == 0
merged_df.loc[mask, 'Start date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)
merged_df.loc[mask, 'Stop date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)

# Calcular a diferença em horas desde o último carregamento
hours_since_last_charge = []
previous_stop_date = None

for index, row in merged_df.iterrows():
    if pd.isnull(row['Start date']) or pd.isnull(row['Stop date']):
        hours_since_last_charge.append(0)
    else:
        if previous_stop_date is None:
            hours_since_last_charge.append(0)
        else:
            diff = (row['Start date'] - previous_stop_date).total_seconds() / 3600
            hours_since_last_charge.append(max(diff, 0))
    previous_stop_date = row['Stop date']

merged_df['Hours_since_last_charge'] = hours_since_last_charge

# Mostrar o DataFrame atualizado
print(merged_df)

          Date          Start date           Stop date  Duration (min)  \
0   2023-02-01 2023-02-01 08:50:00 2023-02-01 12:42:00         232.767   
1   2023-02-02 2023-02-02 00:00:00 2023-02-02 00:00:00           0.000   
2   2023-02-03 2023-02-03 08:08:00 2023-02-03 10:31:00         143.100   
3   2023-02-04 2023-02-04 00:00:00 2023-02-04 00:00:00           0.000   
4   2023-02-05 2023-02-05 00:00:00 2023-02-05 00:00:00           0.000   
..         ...                 ...                 ...             ...   
405 2024-02-24 2024-02-24 00:00:00 2024-02-24 00:00:00           0.000   
406 2024-02-25 2024-02-25 00:00:00 2024-02-25 00:00:00           0.000   
407 2024-02-26 2024-02-26 00:00:00 2024-02-26 00:00:00           0.000   
408 2024-02-27 2024-02-27 00:00:00 2024-02-27 00:00:00           0.000   
409 2024-02-28 2024-02-28 08:36:00 2024-02-28 13:27:00         290.983   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
0                 41.44          

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Definir o range de horas de interesse
#hours = range(8, 21)

# Preparar a matriz de entrada (X) e saída (y)
X = merged_df[['Weekday', 'Hours_since_last_charge']].values
y = merged_df[[f'Charging_{hour}h' for hour in hours]].values

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in hours])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões
print("Previsões:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.ravel(), predictions.ravel())

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')

Acurácia do modelo: 0.9437148217636022
Previsões:
     Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
1   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
2   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
3   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
4   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
..           ...           ...           ...           ...           ...   
77  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
78  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
79  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
80  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
81  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   

    Charging_13h  Charging_14h  Charg

# User 46

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Carregar o dataframe inicial
User = pd.read_excel('EVIO_history_01-02-2023_29-02-2024.xlsx')
User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)', 'Nº cartão EVIO']]

# Remover valores de carregamento de energia inferiores a 1 kWh
User = User[User['Total Energy (kWh)'] >= 1]

# Remover valores de duração de carregamento inferiores a 5 minutos
User = User[User['Duration (min)'] >= 5]

User = User[User['Nº cartão EVIO'] == 46]

# Convertendo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'], format='%m/%d/%Y | %H:%M')
User['Stop date'] = pd.to_datetime(User['Stop date'], format='%m/%d/%Y | %H:%M')

# Criando uma nova coluna 'Weekday' que contém o dia da semana
User['Weekday'] = User['Start date'].dt.day_name()

print(User)

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

User['Weekday'] = User['Weekday'].map(weekday_mapping)

print(User)

              Start date           Stop date  Duration (min)  \
5    2024-02-29 08:57:00 2024-02-29 10:11:00          74.783   
10   2024-02-28 08:49:00 2024-02-28 12:20:00         211.183   
20   2024-02-27 08:50:00 2024-02-27 10:24:00          93.817   
44   2024-02-22 08:43:00 2024-02-22 12:42:00         239.583   
70   2024-02-16 08:39:00 2024-02-16 13:35:00         295.633   
...                  ...                 ...             ...   
1648 2023-03-07 08:36:00 2023-03-07 09:48:00          71.733   
1667 2023-03-03 08:35:00 2023-03-03 09:52:00          77.333   
1691 2023-02-23 08:47:00 2023-02-23 10:36:00         109.450   
1718 2023-02-14 13:20:00 2023-02-14 17:41:00         260.083   
1723 2023-02-14 08:40:00 2023-02-14 10:00:00          79.867   

      Total Energy (kWh) Nº cartão EVIO    Weekday  
5                   9.14             46   Thursday  
10                  8.60             46  Wednesday  
20                  9.57             46    Tuesday  
44                 

In [23]:
# Convertendo as colunas de data para o tipo datetime
#User['Start date'] = pd.to_datetime(User['Start date'])

# Criar uma nova coluna contendo apenas a data (sem a hora)
User['Date'] = User['Start date'].dt.date

# Obter o intervalo de data
start_date = User['Date'].min()
end_date = User['Date'].max()

# Criar um novo DataFrame com todas as datas no intervalo
date_range = pd.date_range(start=start_date, end=end_date)

# Criar um DataFrame vazio para armazenar as datas sem carregamento
empty_dates = pd.DataFrame({'Date': date_range})

#print(User['Date'])
#print(empty_dates, User)

User['Date'] = pd.to_datetime(User['Date'], format='%Y/%m/%d')

#print(empty_dates, User)

"""print(User['Date'].dtype)
print(User.dtype)
print(empty_dates['Date'].dtype)
print(empty_dates.dtype)
"""
# Mesclar os dois DataFrames usando apenas a coluna 'Date'
merged_df = pd.merge(empty_dates, User, on='Date', how='left')

# Preencher NaN e NaT por 0 em todo o DataFrame
merged_df = merged_df.fillna(0)


merged_df['Weekday'] = merged_df['Date'].dt.day_name()

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

merged_df['Weekday'] = merged_df['Weekday'].map(weekday_mapping)

print(merged_df)

          Date           Start date            Stop date  Duration (min)  \
0   2023-02-14  2023-02-14 13:20:00  2023-02-14 17:41:00         260.083   
1   2023-02-14  2023-02-14 08:40:00  2023-02-14 10:00:00          79.867   
2   2023-02-15                    0                    0           0.000   
3   2023-02-16                    0                    0           0.000   
4   2023-02-17                    0                    0           0.000   
..         ...                  ...                  ...             ...   
380 2024-02-25                    0                    0           0.000   
381 2024-02-26                    0                    0           0.000   
382 2024-02-27  2024-02-27 08:50:00  2024-02-27 10:24:00          93.817   
383 2024-02-28  2024-02-28 08:49:00  2024-02-28 12:20:00         211.183   
384 2024-02-29  2024-02-29 08:57:00  2024-02-29 10:11:00          74.783   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0                  4.78             

In [24]:
print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)


# Convertendo as colunas Start date e Stop date para datetime, se necessário
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')


print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)

print(merged_df)


# Criar colunas para cada hora do dia (8h às 20h)
hours = range(8, 21)
for hour in hours:
    merged_df[f'Charging_{hour}h'] = 0

# Preencher as colunas com valores binários (1 ou 0) apenas para as datas presentes no DataFrame original
for index, row in merged_df.iterrows():
    if not pd.isnull(row['Start date']) and not pd.isnull(row['Stop date']):
        start_hour = row['Start date'].hour
        stop_hour = row['Stop date'].hour
        for hour in range(8, 21):
            if hour >= start_hour and hour <= stop_hour:
                merged_df.at[index, f'Charging_{hour}h'] = 1

print(merged_df)

object
object
datetime64[ns]
datetime64[ns]
          Date          Start date           Stop date  Duration (min)  \
0   2023-02-14 2023-02-14 13:20:00 2023-02-14 17:41:00         260.083   
1   2023-02-14 2023-02-14 08:40:00 2023-02-14 10:00:00          79.867   
2   2023-02-15                 NaT                 NaT           0.000   
3   2023-02-16                 NaT                 NaT           0.000   
4   2023-02-17                 NaT                 NaT           0.000   
..         ...                 ...                 ...             ...   
380 2024-02-25                 NaT                 NaT           0.000   
381 2024-02-26                 NaT                 NaT           0.000   
382 2024-02-27 2024-02-27 08:50:00 2024-02-27 10:24:00          93.817   
383 2024-02-28 2024-02-28 08:49:00 2024-02-28 12:20:00         211.183   
384 2024-02-29 2024-02-29 08:57:00 2024-02-29 10:11:00          74.783   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0               

In [25]:
# Ordenar o dataframe pela coluna 'Start date' para garantir que os dados estejam em ordem temporal
merged_df.sort_values(by='Date', inplace=True)


# Converter colunas de data para datetime, tratando '0' como NaT
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

# Atualizar Start date e Stop date onde Total Energy é igual a zero
mask = merged_df['Total Energy (kWh)'] == 0
merged_df.loc[mask, 'Start date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)
merged_df.loc[mask, 'Stop date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)

# Calcular a diferença em horas desde o último carregamento, ignorando os valores NaT
merged_df['Hours_since_last_charge'] = merged_df['Start date'].diff().dt.total_seconds() / 3600

# Substituir NaN com 0 (ou outro valor, se necessário)
merged_df['Hours_since_last_charge'].fillna(0, inplace=True)

# Mostrar o DataFrame atualizado
print(merged_df)

          Date          Start date           Stop date  Duration (min)  \
0   2023-02-14 2023-02-14 13:20:00 2023-02-14 17:41:00         260.083   
1   2023-02-14 2023-02-14 08:40:00 2023-02-14 10:00:00          79.867   
2   2023-02-15 2023-02-15 00:00:00 2023-02-15 00:00:00           0.000   
3   2023-02-16 2023-02-16 00:00:00 2023-02-16 00:00:00           0.000   
4   2023-02-17 2023-02-17 00:00:00 2023-02-17 00:00:00           0.000   
..         ...                 ...                 ...             ...   
380 2024-02-25 2024-02-25 00:00:00 2024-02-25 00:00:00           0.000   
381 2024-02-26 2024-02-26 00:00:00 2024-02-26 00:00:00           0.000   
382 2024-02-27 2024-02-27 08:50:00 2024-02-27 10:24:00          93.817   
383 2024-02-28 2024-02-28 08:49:00 2024-02-28 12:20:00         211.183   
384 2024-02-29 2024-02-29 08:57:00 2024-02-29 10:11:00          74.783   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
0                  4.78          

In [26]:
# Converter colunas de data para datetime, tratando '0' como NaT
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

# Atualizar Start date e Stop date onde Total Energy é igual a zero
mask = merged_df['Total Energy (kWh)'] == 0
merged_df.loc[mask, 'Start date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)
merged_df.loc[mask, 'Stop date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)

# Calcular a diferença em horas desde o último carregamento
hours_since_last_charge = []
previous_stop_date = None

for index, row in merged_df.iterrows():
    if pd.isnull(row['Start date']) or pd.isnull(row['Stop date']):
        hours_since_last_charge.append(0)
    else:
        if previous_stop_date is None:
            hours_since_last_charge.append(0)
        else:
            diff = (row['Start date'] - previous_stop_date).total_seconds() / 3600
            hours_since_last_charge.append(max(diff, 0))
    previous_stop_date = row['Stop date']

merged_df['Hours_since_last_charge'] = hours_since_last_charge

# Mostrar o DataFrame atualizado
print(merged_df)

          Date          Start date           Stop date  Duration (min)  \
0   2023-02-14 2023-02-14 13:20:00 2023-02-14 17:41:00         260.083   
1   2023-02-14 2023-02-14 08:40:00 2023-02-14 10:00:00          79.867   
2   2023-02-15 2023-02-15 00:00:00 2023-02-15 00:00:00           0.000   
3   2023-02-16 2023-02-16 00:00:00 2023-02-16 00:00:00           0.000   
4   2023-02-17 2023-02-17 00:00:00 2023-02-17 00:00:00           0.000   
..         ...                 ...                 ...             ...   
380 2024-02-25 2024-02-25 00:00:00 2024-02-25 00:00:00           0.000   
381 2024-02-26 2024-02-26 00:00:00 2024-02-26 00:00:00           0.000   
382 2024-02-27 2024-02-27 08:50:00 2024-02-27 10:24:00          93.817   
383 2024-02-28 2024-02-28 08:49:00 2024-02-28 12:20:00         211.183   
384 2024-02-29 2024-02-29 08:57:00 2024-02-29 10:11:00          74.783   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
0                  4.78          

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Definir o range de horas de interesse
hours = range(8, 19)

# Preparar a matriz de entrada (X) e saída (y)
X = merged_df[['Weekday', 'Hours_since_last_charge']].values
y = merged_df[[f'Charging_{hour}h' for hour in hours]].values

# Verificar a distribuição das classes em y
class_distribution = {i: np.unique(y[:, i], return_counts=True) for i in range(y.shape[1])}
for i, (classes, counts) in class_distribution.items():
    print(f"Coluna {i}: Classes {classes}, Counts {counts}")

# Remover as colunas de y que contêm apenas uma classe
cols_to_remove = [i for i, (classes, counts) in class_distribution.items() if len(classes) < 2]
print(f"Removendo colunas: {cols_to_remove}")

# Atualizar y removendo as colunas que contêm apenas uma classe
y_filtered = np.delete(y, cols_to_remove, axis=1)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_filtered, test_size=0.2, random_state=42, shuffle=False)

# Verificar se cada coluna de y_train contém mais de uma classe
for i in range(y_train.shape[1]):
    unique_classes = np.unique(y_train[:, i])
    if len(unique_classes) < 2:
        raise ValueError(f"A coluna {i} de y_train contém apenas uma classe. Verifique seus dados.")

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in hours if f'Charging_{hour}h' not in cols_to_remove])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões
print("Previsões:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.ravel(), predictions.ravel())

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')

Coluna 0: Classes [0 1], Counts [297  88]
Coluna 1: Classes [0 1], Counts [290  95]
Coluna 2: Classes [0 1], Counts [294  91]
Coluna 3: Classes [0 1], Counts [305  80]
Coluna 4: Classes [0 1], Counts [308  77]
Coluna 5: Classes [0 1], Counts [311  74]
Coluna 6: Classes [0 1], Counts [361  24]
Coluna 7: Classes [0 1], Counts [369  16]
Coluna 8: Classes [0 1], Counts [373  12]
Coluna 9: Classes [0 1], Counts [377   8]
Coluna 10: Classes [0 1], Counts [381   4]
Removendo colunas: []
Acurácia do modelo: 0.9020070838252656
Previsões:
     Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
1   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
2       Carregar      Carregar      Carregar      Carregar      Carregar   
3   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
4   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
..       

# User 55

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Carregar o dataframe inicial
User = pd.read_excel('EVIO_history_01-02-2023_29-02-2024.xlsx')
User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)', 'Nº cartão EVIO']]

# Remover valores de carregamento de energia inferiores a 1 kWh
User = User[User['Total Energy (kWh)'] >= 1]

# Remover valores de duração de carregamento inferiores a 5 minutos
User = User[User['Duration (min)'] >= 5]

User = User[User['Nº cartão EVIO'] == 55]

# Convertendo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'], format='%m/%d/%Y | %H:%M')
User['Stop date'] = pd.to_datetime(User['Stop date'], format='%m/%d/%Y | %H:%M')

# Criando uma nova coluna 'Weekday' que contém o dia da semana
User['Weekday'] = User['Start date'].dt.day_name()

print(User)

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

User['Weekday'] = User['Weekday'].map(weekday_mapping)

print(User)

              Start date           Stop date  Duration (min)  \
79   2024-02-14 09:03:00 2024-02-14 12:33:00         210.000   
95   2024-02-09 11:33:00 2024-02-09 14:24:00         170.733   
103  2024-02-07 13:05:00 2024-02-07 18:32:00         327.300   
108  2024-02-06 13:35:00 2024-02-06 15:04:00          89.683   
110  2024-02-06 08:51:00 2024-02-06 10:34:00         103.083   
...                  ...                 ...             ...   
1709 2023-02-15 08:58:00 2023-02-15 18:41:00         583.550   
1719 2023-02-14 13:03:00 2023-02-14 18:34:00         331.383   
1735 2023-02-10 08:54:00 2023-02-10 18:36:00         581.350   
1738 2023-02-09 09:01:00 2023-02-09 19:39:00         638.250   
1740 2023-02-08 09:03:00 2023-02-08 18:35:00         571.833   

      Total Energy (kWh) Nº cartão EVIO    Weekday  
79                12.250             55  Wednesday  
95                12.320             55     Friday  
103                5.900             55  Wednesday  
108               1

In [9]:
# Convertendo as colunas de data para o tipo datetime
#User['Start date'] = pd.to_datetime(User['Start date'])

# Criar uma nova coluna contendo apenas a data (sem a hora)
User['Date'] = User['Start date'].dt.date

# Obter o intervalo de data
start_date = User['Date'].min()
end_date = User['Date'].max()

# Criar um novo DataFrame com todas as datas no intervalo
date_range = pd.date_range(start=start_date, end=end_date)

# Criar um DataFrame vazio para armazenar as datas sem carregamento
empty_dates = pd.DataFrame({'Date': date_range})

#print(User['Date'])
#print(empty_dates, User)

User['Date'] = pd.to_datetime(User['Date'], format='%Y/%m/%d')

#print(empty_dates, User)

"""print(User['Date'].dtype)
print(User.dtype)
print(empty_dates['Date'].dtype)
print(empty_dates.dtype)
"""
# Mesclar os dois DataFrames usando apenas a coluna 'Date'
merged_df = pd.merge(empty_dates, User, on='Date', how='left')

# Preencher NaN e NaT por 0 em todo o DataFrame
merged_df = merged_df.fillna(0)


merged_df['Weekday'] = merged_df['Date'].dt.day_name()

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

merged_df['Weekday'] = merged_df['Weekday'].map(weekday_mapping)

print(merged_df)

          Date           Start date            Stop date  Duration (min)  \
0   2023-02-08  2023-02-08 09:03:00  2023-02-08 18:35:00         571.833   
1   2023-02-09  2023-02-09 09:01:00  2023-02-09 19:39:00         638.250   
2   2023-02-10  2023-02-10 08:54:00  2023-02-10 18:36:00         581.350   
3   2023-02-11                    0                    0           0.000   
4   2023-02-12                    0                    0           0.000   
..         ...                  ...                  ...             ...   
400 2024-02-10                    0                    0           0.000   
401 2024-02-11                    0                    0           0.000   
402 2024-02-12                    0                    0           0.000   
403 2024-02-13                    0                    0           0.000   
404 2024-02-14  2024-02-14 09:03:00  2024-02-14 12:33:00         210.000   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0                 12.52             

In [10]:
print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)


# Convertendo as colunas Start date e Stop date para datetime, se necessário
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')


print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)

print(merged_df)


# Criar colunas para cada hora do dia (8h às 20h)
hours = range(8, 21)
for hour in hours:
    merged_df[f'Charging_{hour}h'] = 0

# Preencher as colunas com valores binários (1 ou 0) apenas para as datas presentes no DataFrame original
for index, row in merged_df.iterrows():
    if not pd.isnull(row['Start date']) and not pd.isnull(row['Stop date']):
        start_hour = row['Start date'].hour
        stop_hour = row['Stop date'].hour
        for hour in range(8, 21):
            if hour >= start_hour and hour <= stop_hour:
                merged_df.at[index, f'Charging_{hour}h'] = 1

print(merged_df)

object
object
datetime64[ns]
datetime64[ns]
          Date          Start date           Stop date  Duration (min)  \
0   2023-02-08 2023-02-08 09:03:00 2023-02-08 18:35:00         571.833   
1   2023-02-09 2023-02-09 09:01:00 2023-02-09 19:39:00         638.250   
2   2023-02-10 2023-02-10 08:54:00 2023-02-10 18:36:00         581.350   
3   2023-02-11                 NaT                 NaT           0.000   
4   2023-02-12                 NaT                 NaT           0.000   
..         ...                 ...                 ...             ...   
400 2024-02-10                 NaT                 NaT           0.000   
401 2024-02-11                 NaT                 NaT           0.000   
402 2024-02-12                 NaT                 NaT           0.000   
403 2024-02-13                 NaT                 NaT           0.000   
404 2024-02-14 2024-02-14 09:03:00 2024-02-14 12:33:00         210.000   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0               

In [11]:
# Ordenar o dataframe pela coluna 'Start date' para garantir que os dados estejam em ordem temporal
merged_df.sort_values(by='Date', inplace=True)

# Converter colunas de data para datetime, tratando '0' como NaT
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

# Atualizar Start date e Stop date onde Total Energy é igual a zero
mask = merged_df['Total Energy (kWh)'] == 0
merged_df.loc[mask, 'Start date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)
merged_df.loc[mask, 'Stop date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)

# Calcular a diferença em horas desde o último carregamento, ignorando os valores NaT
merged_df['Hours_since_last_charge'] = merged_df['Start date'].diff().dt.total_seconds() / 3600

# Substituir NaN com 0 (ou outro valor, se necessário)
merged_df['Hours_since_last_charge'].fillna(0, inplace=True)

# Mostrar o DataFrame atualizado
print(merged_df)

          Date          Start date           Stop date  Duration (min)  \
0   2023-02-08 2023-02-08 09:03:00 2023-02-08 18:35:00         571.833   
1   2023-02-09 2023-02-09 09:01:00 2023-02-09 19:39:00         638.250   
2   2023-02-10 2023-02-10 08:54:00 2023-02-10 18:36:00         581.350   
3   2023-02-11 2023-02-11 00:00:00 2023-02-11 00:00:00           0.000   
4   2023-02-12 2023-02-12 00:00:00 2023-02-12 00:00:00           0.000   
..         ...                 ...                 ...             ...   
400 2024-02-10 2024-02-10 00:00:00 2024-02-10 00:00:00           0.000   
401 2024-02-11 2024-02-11 00:00:00 2024-02-11 00:00:00           0.000   
402 2024-02-12 2024-02-12 00:00:00 2024-02-12 00:00:00           0.000   
403 2024-02-13 2024-02-13 00:00:00 2024-02-13 00:00:00           0.000   
404 2024-02-14 2024-02-14 09:03:00 2024-02-14 12:33:00         210.000   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
0                 12.52          

In [12]:
# Converter colunas de data para datetime, tratando '0' como NaT
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

# Atualizar Start date e Stop date onde Total Energy é igual a zero
mask = merged_df['Total Energy (kWh)'] == 0
merged_df.loc[mask, 'Start date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)
merged_df.loc[mask, 'Stop date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)

# Calcular a diferença em horas desde o último carregamento
hours_since_last_charge = []
previous_stop_date = None

for index, row in merged_df.iterrows():
    if pd.isnull(row['Start date']) or pd.isnull(row['Stop date']):
        hours_since_last_charge.append(0)
    else:
        if previous_stop_date is None:
            hours_since_last_charge.append(0)
        else:
            diff = (row['Start date'] - previous_stop_date).total_seconds() / 3600
            hours_since_last_charge.append(max(diff, 0))
    previous_stop_date = row['Stop date']

merged_df['Hours_since_last_charge'] = hours_since_last_charge

# Mostrar o DataFrame atualizado
print(merged_df)

          Date          Start date           Stop date  Duration (min)  \
0   2023-02-08 2023-02-08 09:03:00 2023-02-08 18:35:00         571.833   
1   2023-02-09 2023-02-09 09:01:00 2023-02-09 19:39:00         638.250   
2   2023-02-10 2023-02-10 08:54:00 2023-02-10 18:36:00         581.350   
3   2023-02-11 2023-02-11 00:00:00 2023-02-11 00:00:00           0.000   
4   2023-02-12 2023-02-12 00:00:00 2023-02-12 00:00:00           0.000   
..         ...                 ...                 ...             ...   
400 2024-02-10 2024-02-10 00:00:00 2024-02-10 00:00:00           0.000   
401 2024-02-11 2024-02-11 00:00:00 2024-02-11 00:00:00           0.000   
402 2024-02-12 2024-02-12 00:00:00 2024-02-12 00:00:00           0.000   
403 2024-02-13 2024-02-13 00:00:00 2024-02-13 00:00:00           0.000   
404 2024-02-14 2024-02-14 09:03:00 2024-02-14 12:33:00         210.000   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
0                 12.52          

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Definir o range de horas de interesse
hours = range(8, 19)

# Preparar a matriz de entrada (X) e saída (y)
X = merged_df[['Weekday', 'Hours_since_last_charge']].values
y = merged_df[[f'Charging_{hour}h' for hour in hours]].values

# Verificar a distribuição das classes em y
class_distribution = {i: np.unique(y[:, i], return_counts=True) for i in range(y.shape[1])}
for i, (classes, counts) in class_distribution.items():
    print(f"Coluna {i}: Classes {classes}, Counts {counts}")

# Remover as colunas de y que contêm apenas uma classe
cols_to_remove = [i for i, (classes, counts) in class_distribution.items() if len(classes) < 2]
print(f"Removendo colunas: {cols_to_remove}")

# Atualizar y removendo as colunas que contêm apenas uma classe
y_filtered = np.delete(y, cols_to_remove, axis=1)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_filtered, test_size=0.2, random_state=42, shuffle=False)

# Verificar se cada coluna de y_train contém mais de uma classe
for i in range(y_train.shape[1]):
    unique_classes = np.unique(y_train[:, i])
    if len(unique_classes) < 2:
        raise ValueError(f"A coluna {i} de y_train contém apenas uma classe. Verifique seus dados.")

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in hours if f'Charging_{hour}h' not in cols_to_remove])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões
print("Previsões:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.ravel(), predictions.ravel())

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')

Coluna 0: Classes [0 1], Counts [323  82]
Coluna 1: Classes [0 1], Counts [288 117]
Coluna 2: Classes [0 1], Counts [291 114]
Coluna 3: Classes [0 1], Counts [310  95]
Coluna 4: Classes [0 1], Counts [315  90]
Coluna 5: Classes [0 1], Counts [305 100]
Coluna 6: Classes [0 1], Counts [300 105]
Coluna 7: Classes [0 1], Counts [310  95]
Coluna 8: Classes [0 1], Counts [321  84]
Coluna 9: Classes [0 1], Counts [326  79]
Coluna 10: Classes [0 1], Counts [332  73]
Removendo colunas: []
Acurácia do modelo: 0.8406285072951739
Previsões:
     Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
1   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
2   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
3   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
4   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
..       

# User 76

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Carregar o dataframe inicial
User = pd.read_excel('EVIO_history_01-02-2023_29-02-2024.xlsx')
User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)', 'Nº cartão EVIO']]

# Remover valores de carregamento de energia inferiores a 1 kWh
User = User[User['Total Energy (kWh)'] >= 1]

# Remover valores de duração de carregamento inferiores a 5 minutos
User = User[User['Duration (min)'] >= 5]

User = User[User['Nº cartão EVIO'] == 76]

# Convertendo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'], format='%m/%d/%Y | %H:%M')
User['Stop date'] = pd.to_datetime(User['Stop date'], format='%m/%d/%Y | %H:%M')

# Criando uma nova coluna 'Weekday' que contém o dia da semana
User['Weekday'] = User['Start date'].dt.day_name()

print(User)

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

User['Weekday'] = User['Weekday'].map(weekday_mapping)

print(User)

              Start date           Stop date  Duration (min)  \
79   2024-02-14 09:03:00 2024-02-14 12:33:00         210.000   
95   2024-02-09 11:33:00 2024-02-09 14:24:00         170.733   
103  2024-02-07 13:05:00 2024-02-07 18:32:00         327.300   
108  2024-02-06 13:35:00 2024-02-06 15:04:00          89.683   
110  2024-02-06 08:51:00 2024-02-06 10:34:00         103.083   
...                  ...                 ...             ...   
1709 2023-02-15 08:58:00 2023-02-15 18:41:00         583.550   
1719 2023-02-14 13:03:00 2023-02-14 18:34:00         331.383   
1735 2023-02-10 08:54:00 2023-02-10 18:36:00         581.350   
1738 2023-02-09 09:01:00 2023-02-09 19:39:00         638.250   
1740 2023-02-08 09:03:00 2023-02-08 18:35:00         571.833   

      Total Energy (kWh) Nº cartão EVIO    Weekday  
79                12.250             55  Wednesday  
95                12.320             55     Friday  
103                5.900             55  Wednesday  
108               1

In [15]:
# Convertendo as colunas de data para o tipo datetime
#User['Start date'] = pd.to_datetime(User['Start date'])

# Criar uma nova coluna contendo apenas a data (sem a hora)
User['Date'] = User['Start date'].dt.date

# Obter o intervalo de data
start_date = User['Date'].min()
end_date = User['Date'].max()

# Criar um novo DataFrame com todas as datas no intervalo
date_range = pd.date_range(start=start_date, end=end_date)

# Criar um DataFrame vazio para armazenar as datas sem carregamento
empty_dates = pd.DataFrame({'Date': date_range})

#print(User['Date'])
#print(empty_dates, User)

User['Date'] = pd.to_datetime(User['Date'], format='%Y/%m/%d')

#print(empty_dates, User)

"""print(User['Date'].dtype)
print(User.dtype)
print(empty_dates['Date'].dtype)
print(empty_dates.dtype)
"""
# Mesclar os dois DataFrames usando apenas a coluna 'Date'
merged_df = pd.merge(empty_dates, User, on='Date', how='left')

# Preencher NaN e NaT por 0 em todo o DataFrame
merged_df = merged_df.fillna(0)


merged_df['Weekday'] = merged_df['Date'].dt.day_name()

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

merged_df['Weekday'] = merged_df['Weekday'].map(weekday_mapping)

print(merged_df)

          Date           Start date            Stop date  Duration (min)  \
0   2023-02-08  2023-02-08 09:03:00  2023-02-08 18:35:00         571.833   
1   2023-02-09  2023-02-09 09:01:00  2023-02-09 19:39:00         638.250   
2   2023-02-10  2023-02-10 08:54:00  2023-02-10 18:36:00         581.350   
3   2023-02-11                    0                    0           0.000   
4   2023-02-12                    0                    0           0.000   
..         ...                  ...                  ...             ...   
389 2024-02-10                    0                    0           0.000   
390 2024-02-11                    0                    0           0.000   
391 2024-02-12                    0                    0           0.000   
392 2024-02-13                    0                    0           0.000   
393 2024-02-14  2024-02-14 09:03:00  2024-02-14 12:33:00         210.000   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0                 12.52             

In [16]:
print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)


# Convertendo as colunas Start date e Stop date para datetime, se necessário
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')


print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)

print(merged_df)


# Criar colunas para cada hora do dia (8h às 20h)
hours = range(8, 21)
for hour in hours:
    merged_df[f'Charging_{hour}h'] = 0

# Preencher as colunas com valores binários (1 ou 0) apenas para as datas presentes no DataFrame original
for index, row in merged_df.iterrows():
    if not pd.isnull(row['Start date']) and not pd.isnull(row['Stop date']):
        start_hour = row['Start date'].hour
        stop_hour = row['Stop date'].hour
        for hour in range(8, 21):
            if hour >= start_hour and hour <= stop_hour:
                merged_df.at[index, f'Charging_{hour}h'] = 1

print(merged_df)

object
object
datetime64[ns]
datetime64[ns]
          Date          Start date           Stop date  Duration (min)  \
0   2023-02-08 2023-02-08 09:03:00 2023-02-08 18:35:00         571.833   
1   2023-02-09 2023-02-09 09:01:00 2023-02-09 19:39:00         638.250   
2   2023-02-10 2023-02-10 08:54:00 2023-02-10 18:36:00         581.350   
3   2023-02-11                 NaT                 NaT           0.000   
4   2023-02-12                 NaT                 NaT           0.000   
..         ...                 ...                 ...             ...   
389 2024-02-10                 NaT                 NaT           0.000   
390 2024-02-11                 NaT                 NaT           0.000   
391 2024-02-12                 NaT                 NaT           0.000   
392 2024-02-13                 NaT                 NaT           0.000   
393 2024-02-14 2024-02-14 09:03:00 2024-02-14 12:33:00         210.000   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0               

In [17]:
# Ordenar o dataframe pela coluna 'Start date' para garantir que os dados estejam em ordem temporal
merged_df.sort_values(by='Date', inplace=True)


# Converter colunas de data para datetime, tratando '0' como NaT
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

# Atualizar Start date e Stop date onde Total Energy é igual a zero
mask = merged_df['Total Energy (kWh)'] == 0
merged_df.loc[mask, 'Start date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)
merged_df.loc[mask, 'Stop date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)

# Calcular a diferença em horas desde o último carregamento, ignorando os valores NaT
merged_df['Hours_since_last_charge'] = merged_df['Start date'].diff().dt.total_seconds() / 3600

# Substituir NaN com 0 (ou outro valor, se necessário)
merged_df['Hours_since_last_charge'].fillna(0, inplace=True)

# Mostrar o DataFrame atualizado
print(merged_df)

          Date          Start date           Stop date  Duration (min)  \
0   2023-02-08 2023-02-08 09:03:00 2023-02-08 18:35:00         571.833   
1   2023-02-09 2023-02-09 09:01:00 2023-02-09 19:39:00         638.250   
2   2023-02-10 2023-02-10 08:54:00 2023-02-10 18:36:00         581.350   
3   2023-02-11 2023-02-11 00:00:00 2023-02-11 00:00:00           0.000   
4   2023-02-12 2023-02-12 00:00:00 2023-02-12 00:00:00           0.000   
..         ...                 ...                 ...             ...   
389 2024-02-10 2024-02-10 00:00:00 2024-02-10 00:00:00           0.000   
390 2024-02-11 2024-02-11 00:00:00 2024-02-11 00:00:00           0.000   
391 2024-02-12 2024-02-12 00:00:00 2024-02-12 00:00:00           0.000   
392 2024-02-13 2024-02-13 00:00:00 2024-02-13 00:00:00           0.000   
393 2024-02-14 2024-02-14 09:03:00 2024-02-14 12:33:00         210.000   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
0                 12.52          

In [18]:
# Converter colunas de data para datetime, tratando '0' como NaT
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

# Atualizar Start date e Stop date onde Total Energy é igual a zero
mask = merged_df['Total Energy (kWh)'] == 0
merged_df.loc[mask, 'Start date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)
merged_df.loc[mask, 'Stop date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)

# Calcular a diferença em horas desde o último carregamento
hours_since_last_charge = []
previous_stop_date = None

for index, row in merged_df.iterrows():
    if pd.isnull(row['Start date']) or pd.isnull(row['Stop date']):
        hours_since_last_charge.append(0)
    else:
        if previous_stop_date is None:
            hours_since_last_charge.append(0)
        else:
            diff = (row['Start date'] - previous_stop_date).total_seconds() / 3600
            hours_since_last_charge.append(max(diff, 0))
    previous_stop_date = row['Stop date']

merged_df['Hours_since_last_charge'] = hours_since_last_charge

# Mostrar o DataFrame atualizado
print(merged_df)

          Date          Start date           Stop date  Duration (min)  \
0   2023-02-08 2023-02-08 09:03:00 2023-02-08 18:35:00         571.833   
1   2023-02-09 2023-02-09 09:01:00 2023-02-09 19:39:00         638.250   
2   2023-02-10 2023-02-10 08:54:00 2023-02-10 18:36:00         581.350   
3   2023-02-11 2023-02-11 00:00:00 2023-02-11 00:00:00           0.000   
4   2023-02-12 2023-02-12 00:00:00 2023-02-12 00:00:00           0.000   
..         ...                 ...                 ...             ...   
389 2024-02-10 2024-02-10 00:00:00 2024-02-10 00:00:00           0.000   
390 2024-02-11 2024-02-11 00:00:00 2024-02-11 00:00:00           0.000   
391 2024-02-12 2024-02-12 00:00:00 2024-02-12 00:00:00           0.000   
392 2024-02-13 2024-02-13 00:00:00 2024-02-13 00:00:00           0.000   
393 2024-02-14 2024-02-14 09:03:00 2024-02-14 12:33:00         210.000   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
0                 12.52          

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Definir o range de horas de interesse
hours = range(8, 19)

# Preparar a matriz de entrada (X) e saída (y)
X = merged_df[['Weekday', 'Hours_since_last_charge']].values
y = merged_df[[f'Charging_{hour}h' for hour in hours]].values

# Verificar a distribuição das classes em y
class_distribution = {i: np.unique(y[:, i], return_counts=True) for i in range(y.shape[1])}
for i, (classes, counts) in class_distribution.items():
    print(f"Coluna {i}: Classes {classes}, Counts {counts}")

# Remover as colunas de y que contêm apenas uma classe
cols_to_remove = [i for i, (classes, counts) in class_distribution.items() if len(classes) < 2]
print(f"Removendo colunas: {cols_to_remove}")

# Atualizar y removendo as colunas que contêm apenas uma classe
y_filtered = np.delete(y, cols_to_remove, axis=1)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_filtered, test_size=0.2, random_state=42, shuffle=False)

# Verificar se cada coluna de y_train contém mais de uma classe
for i in range(y_train.shape[1]):
    unique_classes = np.unique(y_train[:, i])
    if len(unique_classes) < 2:
        raise ValueError(f"A coluna {i} de y_train contém apenas uma classe. Verifique seus dados.")

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in hours if f'Charging_{hour}h' not in cols_to_remove])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões
print("Previsões:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.ravel(), predictions.ravel())

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')

Coluna 0: Classes [0 1], Counts [316  78]
Coluna 1: Classes [0 1], Counts [281 113]
Coluna 2: Classes [0 1], Counts [281 113]
Coluna 3: Classes [0 1], Counts [300  94]
Coluna 4: Classes [0 1], Counts [305  89]
Coluna 5: Classes [0 1], Counts [295  99]
Coluna 6: Classes [0 1], Counts [295  99]
Coluna 7: Classes [0 1], Counts [302  92]
Coluna 8: Classes [0 1], Counts [312  82]
Coluna 9: Classes [0 1], Counts [316  78]
Coluna 10: Classes [0 1], Counts [321  73]
Removendo colunas: []
Acurácia do modelo: 0.8331415420023015
Previsões:
     Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0   Não Carregar      Carregar  Não Carregar  Não Carregar  Não Carregar   
1   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
2   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
3   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
4   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
..       

# User 144

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Carregar o dataframe inicial
User = pd.read_excel('EVIO_history_01-02-2023_29-02-2024.xlsx')
User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)', 'Nº cartão EVIO']]

# Remover valores de carregamento de energia inferiores a 1 kWh
User = User[User['Total Energy (kWh)'] >= 1]

# Remover valores de duração de carregamento inferiores a 5 minutos
User = User[User['Duration (min)'] >= 5]

User = User[User['Nº cartão EVIO'] == 144]

# Convertendo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'], format='%m/%d/%Y | %H:%M')
User['Stop date'] = pd.to_datetime(User['Stop date'], format='%m/%d/%Y | %H:%M')

# Criando uma nova coluna 'Weekday' que contém o dia da semana
User['Weekday'] = User['Start date'].dt.day_name()

print(User)

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

User['Weekday'] = User['Weekday'].map(weekday_mapping)

print(User)

              Start date           Stop date  Duration (min)  \
1    2024-02-29 10:44:00 2024-02-29 11:24:00          39.483   
15   2024-02-27 16:35:00 2024-02-27 17:07:00          32.583   
37   2024-02-23 08:24:00 2024-02-23 09:49:00          85.183   
48   2024-02-21 13:14:00 2024-02-21 14:06:00          51.883   
78   2024-02-14 10:18:00 2024-02-14 11:05:00          46.283   
...                  ...                 ...             ...   
1603 2023-03-17 14:07:00 2023-03-17 15:39:00          92.517   
1629 2023-03-10 08:42:00 2023-03-10 11:24:00         161.967   
1639 2023-03-08 14:03:00 2023-03-08 17:14:00         190.867   
1651 2023-03-06 13:35:00 2023-03-06 17:46:00         250.533   
1662 2023-03-03 21:59:00 2023-03-03 22:52:00          52.567   

      Total Energy (kWh) Nº cartão EVIO    Weekday  
1                  19.95            144   Thursday  
15                 29.61            144    Tuesday  
37                 14.47            144     Friday  
48                 

In [22]:
# Convertendo as colunas de data para o tipo datetime
#User['Start date'] = pd.to_datetime(User['Start date'])

# Criar uma nova coluna contendo apenas a data (sem a hora)
User['Date'] = User['Start date'].dt.date

# Obter o intervalo de data
start_date = User['Date'].min()
end_date = User['Date'].max()

# Criar um novo DataFrame com todas as datas no intervalo
date_range = pd.date_range(start=start_date, end=end_date)

# Criar um DataFrame vazio para armazenar as datas sem carregamento
empty_dates = pd.DataFrame({'Date': date_range})

#print(User['Date'])
#print(empty_dates, User)

User['Date'] = pd.to_datetime(User['Date'], format='%Y/%m/%d')

#print(empty_dates, User)

"""print(User['Date'].dtype)
print(User.dtype)
print(empty_dates['Date'].dtype)
print(empty_dates.dtype)
"""
# Mesclar os dois DataFrames usando apenas a coluna 'Date'
merged_df = pd.merge(empty_dates, User, on='Date', how='left')

# Preencher NaN e NaT por 0 em todo o DataFrame
merged_df = merged_df.fillna(0)


merged_df['Weekday'] = merged_df['Date'].dt.day_name()

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

merged_df['Weekday'] = merged_df['Weekday'].map(weekday_mapping)

print(merged_df)

          Date           Start date            Stop date  Duration (min)  \
0   2023-03-03  2023-03-03 21:59:00  2023-03-03 22:52:00          52.567   
1   2023-03-04                    0                    0           0.000   
2   2023-03-05                    0                    0           0.000   
3   2023-03-06  2023-03-06 13:35:00  2023-03-06 17:46:00         250.533   
4   2023-03-07                    0                    0           0.000   
..         ...                  ...                  ...             ...   
370 2024-02-25                    0                    0           0.000   
371 2024-02-26                    0                    0           0.000   
372 2024-02-27  2024-02-27 16:35:00  2024-02-27 17:07:00          32.583   
373 2024-02-28                    0                    0           0.000   
374 2024-02-29  2024-02-29 10:44:00  2024-02-29 11:24:00          39.483   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0                 46.92             

In [23]:
print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)


# Convertendo as colunas Start date e Stop date para datetime, se necessário
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')


print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)

print(merged_df)


# Criar colunas para cada hora do dia (8h às 20h)
hours = range(8, 21)
for hour in hours:
    merged_df[f'Charging_{hour}h'] = 0

# Preencher as colunas com valores binários (1 ou 0) apenas para as datas presentes no DataFrame original
for index, row in merged_df.iterrows():
    if not pd.isnull(row['Start date']) and not pd.isnull(row['Stop date']):
        start_hour = row['Start date'].hour
        stop_hour = row['Stop date'].hour
        for hour in range(8, 21):
            if hour >= start_hour and hour <= stop_hour:
                merged_df.at[index, f'Charging_{hour}h'] = 1

print(merged_df)

object
object
datetime64[ns]
datetime64[ns]
          Date          Start date           Stop date  Duration (min)  \
0   2023-03-03 2023-03-03 21:59:00 2023-03-03 22:52:00          52.567   
1   2023-03-04                 NaT                 NaT           0.000   
2   2023-03-05                 NaT                 NaT           0.000   
3   2023-03-06 2023-03-06 13:35:00 2023-03-06 17:46:00         250.533   
4   2023-03-07                 NaT                 NaT           0.000   
..         ...                 ...                 ...             ...   
370 2024-02-25                 NaT                 NaT           0.000   
371 2024-02-26                 NaT                 NaT           0.000   
372 2024-02-27 2024-02-27 16:35:00 2024-02-27 17:07:00          32.583   
373 2024-02-28                 NaT                 NaT           0.000   
374 2024-02-29 2024-02-29 10:44:00 2024-02-29 11:24:00          39.483   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0               

In [24]:
# Ordenar o dataframe pela coluna 'Start date' para garantir que os dados estejam em ordem temporal
merged_df.sort_values(by='Date', inplace=True)


# Converter colunas de data para datetime, tratando '0' como NaT
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

# Atualizar Start date e Stop date onde Total Energy é igual a zero
mask = merged_df['Total Energy (kWh)'] == 0
merged_df.loc[mask, 'Start date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)
merged_df.loc[mask, 'Stop date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)

# Calcular a diferença em horas desde o último carregamento, ignorando os valores NaT
merged_df['Hours_since_last_charge'] = merged_df['Start date'].diff().dt.total_seconds() / 3600

# Substituir NaN com 0 (ou outro valor, se necessário)
merged_df['Hours_since_last_charge'].fillna(0, inplace=True)

# Mostrar o DataFrame atualizado
print(merged_df)

          Date          Start date           Stop date  Duration (min)  \
0   2023-03-03 2023-03-03 21:59:00 2023-03-03 22:52:00          52.567   
1   2023-03-04 2023-03-04 00:00:00 2023-03-04 00:00:00           0.000   
2   2023-03-05 2023-03-05 00:00:00 2023-03-05 00:00:00           0.000   
3   2023-03-06 2023-03-06 13:35:00 2023-03-06 17:46:00         250.533   
4   2023-03-07 2023-03-07 00:00:00 2023-03-07 00:00:00           0.000   
..         ...                 ...                 ...             ...   
370 2024-02-25 2024-02-25 00:00:00 2024-02-25 00:00:00           0.000   
371 2024-02-26 2024-02-26 00:00:00 2024-02-26 00:00:00           0.000   
372 2024-02-27 2024-02-27 16:35:00 2024-02-27 17:07:00          32.583   
373 2024-02-28 2024-02-28 00:00:00 2024-02-28 00:00:00           0.000   
374 2024-02-29 2024-02-29 10:44:00 2024-02-29 11:24:00          39.483   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
0                 46.92          

In [25]:
# Converter colunas de data para datetime, tratando '0' como NaT
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

# Atualizar Start date e Stop date onde Total Energy é igual a zero
mask = merged_df['Total Energy (kWh)'] == 0
merged_df.loc[mask, 'Start date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)
merged_df.loc[mask, 'Stop date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)

# Calcular a diferença em horas desde o último carregamento
hours_since_last_charge = []
previous_stop_date = None

for index, row in merged_df.iterrows():
    if pd.isnull(row['Start date']) or pd.isnull(row['Stop date']):
        hours_since_last_charge.append(0)
    else:
        if previous_stop_date is None:
            hours_since_last_charge.append(0)
        else:
            diff = (row['Start date'] - previous_stop_date).total_seconds() / 3600
            hours_since_last_charge.append(max(diff, 0))
    previous_stop_date = row['Stop date']

merged_df['Hours_since_last_charge'] = hours_since_last_charge

# Mostrar o DataFrame atualizado
print(merged_df)

          Date          Start date           Stop date  Duration (min)  \
0   2023-03-03 2023-03-03 21:59:00 2023-03-03 22:52:00          52.567   
1   2023-03-04 2023-03-04 00:00:00 2023-03-04 00:00:00           0.000   
2   2023-03-05 2023-03-05 00:00:00 2023-03-05 00:00:00           0.000   
3   2023-03-06 2023-03-06 13:35:00 2023-03-06 17:46:00         250.533   
4   2023-03-07 2023-03-07 00:00:00 2023-03-07 00:00:00           0.000   
..         ...                 ...                 ...             ...   
370 2024-02-25 2024-02-25 00:00:00 2024-02-25 00:00:00           0.000   
371 2024-02-26 2024-02-26 00:00:00 2024-02-26 00:00:00           0.000   
372 2024-02-27 2024-02-27 16:35:00 2024-02-27 17:07:00          32.583   
373 2024-02-28 2024-02-28 00:00:00 2024-02-28 00:00:00           0.000   
374 2024-02-29 2024-02-29 10:44:00 2024-02-29 11:24:00          39.483   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
0                 46.92          

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Definir o range de horas de interesse
hours = range(8, 20)

# Preparar a matriz de entrada (X) e saída (y)
X = merged_df[['Weekday', 'Hours_since_last_charge']].values
y = merged_df[[f'Charging_{hour}h' for hour in hours]].values

# Verificar a distribuição das classes em y
class_distribution = {i: np.unique(y[:, i], return_counts=True) for i in range(y.shape[1])}
for i, (classes, counts) in class_distribution.items():
    print(f"Coluna {i}: Classes {classes}, Counts {counts}")

# Remover as colunas de y que contêm apenas uma classe
cols_to_remove = [i for i, (classes, counts) in class_distribution.items() if len(classes) < 2]
print(f"Removendo colunas: {cols_to_remove}")

# Atualizar y removendo as colunas que contêm apenas uma classe
y_filtered = np.delete(y, cols_to_remove, axis=1)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_filtered, test_size=0.2, random_state=42, shuffle=False)

# Verificar se cada coluna de y_train contém mais de uma classe
for i in range(y_train.shape[1]):
    unique_classes = np.unique(y_train[:, i])
    if len(unique_classes) < 2:
        raise ValueError(f"A coluna {i} de y_train contém apenas uma classe. Verifique seus dados.")

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in hours if f'Charging_{hour}h' not in cols_to_remove])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões
print("Previsões:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.ravel(), predictions.ravel())

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')

Coluna 0: Classes [0 1], Counts [337  38]
Coluna 1: Classes [0 1], Counts [334  41]
Coluna 2: Classes [0 1], Counts [333  42]
Coluna 3: Classes [0 1], Counts [328  47]
Coluna 4: Classes [0 1], Counts [340  35]
Coluna 5: Classes [0 1], Counts [350  25]
Coluna 6: Classes [0 1], Counts [350  25]
Coluna 7: Classes [0 1], Counts [358  17]
Coluna 8: Classes [0 1], Counts [357  18]
Coluna 9: Classes [0 1], Counts [359  16]
Coluna 10: Classes [0 1], Counts [370   5]
Coluna 11: Classes [0 1], Counts [371   4]
Removendo colunas: []
Acurácia do modelo: 0.9333333333333333
Previsões:
     Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
1   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
2   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
3   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
4   Não Carregar  Não Carregar  Não Carreg